In [126]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from skimage.feature import hog
from sklearn.svm import LinearSVC
from skimage.io import imread
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import accuracy_score, fbeta_score, precision_score, recall_score


In [127]:
def num_images(image_path):
    count = 0
    for dir1 in os.listdir(image_path):
        count += len(os.listdir(os.path.join(image_path, dir1)))
    return count

In [128]:
# split data
split_ratio = math.floor(0.8 * num_images('train_images'))

data = pd.read_csv('./train.csv')
data = data[['video_id', 'video_frame', 'annotations']]
data["annotations"] = data["annotations"].map(lambda x: eval(x))

In [129]:
data['count'] = 0
for i in range(0, len(data['annotations'])):
    data['count'][i] = len(data['annotations'][i])
data

/mnt/c/Users/pokev/OneDrive/Documents/Uni/COMP9517/Crown-of-Throrns/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,video_id,video_frame,annotations,count
0,0,0,[],0
1,0,1,[],0
2,0,2,[],0
3,0,3,[],0
4,0,4,[],0
...,...,...,...,...
23496,2,10755,[],0
23497,2,10756,[],0
23498,2,10757,[],0
23499,2,10758,[],0


In [130]:
training_df = data[0:split_ratio]
testing_df = data[split_ratio:]
training_df

,video_id,video_frame,annotations,count
0,0,0,[],0
1,0,1,[],0
2,0,2,[],0
3,0,3,[],0
4,0,4,[],0
...,...,...,...,...
18795,2,4727,"[{'x': 148, 'y': 521, 'width': 45, 'height': 48}]",1
18796,2,4728,"[{'x': 134, 'y': 518, 'width': 45, 'height': 48}]",1
18797,2,4729,"[{'x': 120, 'y': 515, 'width': 45, 'height': 48}]",1
18798,2,4730,"[{'x': 114, 'y': 511, 'width': 45, 'height': 48}]",1


In [131]:
window_size = (64,128)

cots_df = []
no_cots_df = []

cots_features = []
no_cots_features = []

cots_labels = []
no_cots_labels = []

# for i in range(0, len(training_df['video_id'])):
for i in range(0, 100):
    video_id = str(training_df['video_id'][i])
    frame_id = str(training_df['video_frame'][i])
    path = 'train_images/video_'+video_id+'/'+frame_id+'.jpg'
    img = cv2.imread(path, cv2.COLOR_BGR2RGB)
    
    if training_df['count'][i] == 0:
        # If no COTS in image
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, window_size)
        no_cots_df.append(img)
        feature = hog(img, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), visualize=False, feature_vector=True)
        no_cots_features.append(feature)
        no_cots_labels.append(0)
    else:
        # If COTS is in image
        for j in training_df['annotations'][i]:
            cots = img[j['y']:j['y']+j['height'], j['x']:j['x']+j['width']]
            cots = cv2.cvtColor(cots, cv2.COLOR_RGB2GRAY)
            cots = cv2.resize(cots, window_size)
            cots_df.append(cots)
            feature = hog(cots, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), visualize=False, feature_vector=True)
            cots_features.append(feature)
            cots_labels.append(training_df['count'][i])


In [132]:
x = np.asarray(cots_features + no_cots_features)
y = np.asarray(cots_labels + no_cots_labels)

In [133]:
model = LinearSVC()
model.fit(x,y)
y_pred = svm_classifier.predict(x_test)